In [ ]:
"""
A notebook to showcase my DQN,
it's now able to be trained to win a small tournament against TFT and Alternator
Sorry for the mess as I'm actively developing the codes
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import gc
import copy
import random
import numpy as np
import axelrod as axl
from pprint import pprint
from itertools import permutations
from collections import namedtuple, deque

np.set_printoptions(precision=2)

# the following imports are from network / axl_utils folder
import network
from network.dqn import DQN
from network.replay import ReplayMemory
from axl_utils.nnplayer import NNplayer, State

In [66]:
GAME_LEN = 20 + 1
C = axl.Action.C
D = axl.Action.D
GAME = axl.Game(r=3, s=0, t=5, p=1)

def Match(players, turns=GAME_LEN, reset=True):
    return axl.Match(players, turns=turns, reset=reset, game=GAME)

def play(player1, player2, match):
    game = match((player1, player2))
    actions = game.play()
    scores = game.scores()[:-1]
    print(scores)
    print(f"Player 1 score = {sum(list(zip(*scores))[0])}")
    print(f"Player 2 score = {sum(list(zip(*scores))[1])}")
    return game

In [12]:
play(axl.Alternator(), axl.Random(), Match)
game = play(axl.Prober4(), axl.TitForTat(), Match)

[(C, D), (D, C), (C, C), (D, C), (C, C), (D, D), (C, C), (D, C), (C, D), (D, C), (C, D), (D, C), (C, C), (D, C), (C, D), (D, D), (C, C), (D, D), (C, D), (D, C), (C, C)]
Player 1 score = 53
Player 2 score = 43
[(C, C), (C, C), (D, C), (C, D), (D, C), (D, D), (D, D), (C, D), (C, C), (D, C), (C, D), (D, C), (C, D), (C, C), (D, C), (C, D), (D, C), (D, D), (C, D), (D, C), (C, D)]
Player 1 score = 50
Player 2 score = 45


In [13]:
s = State(GAME_LEN)
print(s.values())

s.push(C, D)
print(s)

[[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]]
[deque([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1], maxlen=21),
 deque([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0], maxlen=21)]


In [124]:
dqn = DQN([
#                     network.Flatten_layer(),
#                     network.Maxout_layer(GAME_LEN*2, 100),
#                     network.Maxout_layer(100, 40),
#                     network.Maxout_layer(40, 20),
#                     network.Maxout_layer(20, 2),
    
                    network.Flatten_layer(),
                    network.Linear_layer(GAME_LEN*2, 100, bias=0.01),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(100, 200),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(200, 100),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(100, 40),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(40, 2),
                    ])
p1 = NNplayer(dqn, ReplayMemory(2000), State(GAME_LEN), gamma=0.9)
del dqn
gc.collect()

0

In [125]:
param = {"lr": 1e-5, 'batch': 16, "momentum": 0.9, "mode": "train", "eps": 1e-16, "beta":(0.9, 0.999), 
         "epoch": 0, 'optimizer': 'adam', 't': 1, 'clip': 1.0, 'decay': 0.0}

In [126]:
# enable test mode
with p1:
    players = (p1, axl.TitForTat())
    game = Match(players)
    game.play()
    
# initial Q_values:

[[2.28 0.41]]
[[1.09 0.86]]
[[1.01 0.25]]
[[ 0.45 -0.31]]
[[-0.22  0.03]]
[[9.74e-01 4.25e-04]]
[[0.51 0.84]]
[[ 1.48 -0.48]]
[[ 0.78 -0.86]]
[[ 0.28 -0.05]]
[[ 0.72 -1.37]]
[[ 0.   -0.65]]
[[-0.22 -0.8 ]]
[[-0.62  1.45]]
[[1.06 1.75]]
[[0.05 2.93]]
[[0.58 1.99]]
[[0.7  1.87]]
[[0.26 1.65]]
[[0.63 1.88]]
[[0.26 1.08]]


In [127]:
# initial experience
# high greedy -> basically random exploration
p1.greedy = 0.5
for i in range(60):
    players = (p1, axl.TitForTat())
    game = Match(players)
    game.play()
    
    players = (p1, axl.Alternator())
    game = Match(players)
    game.play()
p1.greedy = 0.2

In [128]:
for _ in range(50):
    p1.train(80, param)
    [Match((p1, axl.TitForTat())).play() for _ in range(10)]
    [Match((p1, axl.Alternator())).play() for _ in range(10)]
    print(f'{p1.network.loss:.4f}')

0.7017
0.5120
0.5724
0.5175
0.4411
0.4019
0.4471
0.4050
0.3844
0.3372
0.3235
0.2613
0.3243
0.2961
0.2521
0.3112
0.2155
0.1990
0.2559
0.2065
0.1865
0.2101
0.1880
0.1632
0.1521
0.1404
0.1352
0.2555
0.1764
0.1510
0.1530


KeyboardInterrupt: 

In [129]:
with p1:
    play(p1, axl.TitForTat(), Match)

[[25.47 26.25]]
[[22.6 23.4]]
[[22.9  24.37]]
[[22.29 20.79]]
[[24.82 23.61]]
[[24.31 23.49]]
[[23.7  22.93]]
[[22.95 22.17]]
[[22.16 21.35]]
[[21.32 20.57]]
[[20.4  19.44]]
[[19.31 18.51]]
[[18.12 17.21]]
[[16.78 16.04]]
[[15.23 13.98]]
[[13.65 12.74]]
[[11.84 10.87]]
[[9.89 9.12]]
[[7.58 5.81]]
[[3.26 5.03]]
[[-0.   -0.03]]
[(5, 0), (1, 1), (1, 1), (0, 5), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (5, 0)]
Player 1 score = 57
Player 2 score = 52


In [130]:
with p1:
    play(p1, axl.Alternator(), Match)

[[25.47 26.25]]
[[22.6 23.4]]
[[22.9  24.37]]
[[22.46 23.69]]
[[23.32 25.2 ]]
[[21.4  22.51]]
[[21.85 23.88]]
[[19.86 21.02]]
[[20.09 22.23]]
[[18.3  19.17]]
[[18.33 20.16]]
[[16.11 16.9 ]]
[[15.52 17.72]]
[[13.17 14.15]]
[[12.55 14.58]]
[[ 9.59 10.62]]
[[ 8.55 10.65]]
[[5.28 6.28]]
[[3.84 5.93]]
[[-0.01  1.  ]]
[[ 0.01 -0.17]]
[(5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1), (5, 0), (1, 1)]
Player 1 score = 60
Player 2 score = 10


In [123]:
play(axl.TitForTat(), axl.Alternator(), Match)

[(3, 3), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5), (5, 0), (0, 5)]
Player 1 score = 48
Player 2 score = 53


In [46]:
# theoritical Q values
# assuming both player cooperate to the end
for i in range(20):
    print(f'{3.0+3.0/(1/9) * (1.0 - 1.0/(1.0+1/9)**(19-i)):.2f}')

26.35
25.95
25.50
25.00
24.44
23.82
23.14
22.37
21.53
20.59
19.54
18.38
17.09
15.65
14.06
12.29
10.32
8.13
5.70
3.00


In [28]:
p1.network.policy_net.print_parameters()

--0--
Printing flatten layer:
{'shape': (1, 2, 21), 'type': 'flatten'}
--1--
Printing linear layer:
{'bias': 0.01,
 'input': array([[-1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., -1.,  1.,  0.,  1.,  0.,
         1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,
         0.,  1.,  0.,  1.]]),
 'input_nodes': 42,
 'm1': array([[ 0.01, -0.01,  0.  , ..., -0.  ,  0.  ,  0.01],
       [ 0.01, -0.01,  0.  , ..., -0.  ,  0.  ,  0.01],
       [ 0.01, -0.01,  0.01, ..., -0.  ,  0.  ,  0.01],
       ...,
       [-0.01,  0.01, -0.  , ...,  0.  , -0.  ,  0.  ],
       [-0.01,  0.01,  0.  , ..., -0.  ,  0.  ,  0.  ],
       [-0.01,  0.01, -0.  , ...,  0.  , -0.  , -0.01]]),
 'm2': array([[0.01, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.01, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.01, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ...